In [37]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import xarray as xr
import requests
import netCDF4
import boto3
import boto3
from botocore import UNSIGNED
from botocore.config import Config
import os
from tqdm import tqdm


%matplotlib inline


In [38]:
bucket_name = 'noaa-goes17'
product_name = 'GLM-L2-LCFA'
year = 2020
day_of_year = 229
hour = 1
#band = 8

s3_client = boto3.client('s3', config=Config(signature_version=UNSIGNED))
def get_s3_keys(bucket, s3_client, prefix = ''):
    """
    Generate the keys in an S3 bucket.

    :param bucket: Name of the S3 bucket.
    :param prefix: Only fetch keys that start with this prefix (optional).
    """
    
    kwargs = {'Bucket': bucket}

    if isinstance(prefix, str):
        kwargs['Prefix'] = prefix

    while True:
        resp = s3_client.list_objects_v2(**kwargs)
        for obj in resp['Contents']:
            key = obj['Key']
            if key.startswith(prefix):
                yield key

        try:
            kwargs['ContinuationToken'] = resp['NextContinuationToken']
        except KeyError:
            break



In [39]:
def load_file_names(selection_string = "", selection_string2 = "", avoid_string = "",folder_list = []):
    for area_name in folder_list:
        selection_string = selection_string
        selection_string2 = selection_string2
        avoid_string = avoid_string
        filenamelist = []
        #area_name = "./Area-4"
        for root, dirs, files in os.walk(area_name, topdown=False):
            for name in files:
                #print(os.path.join(root, name))
                temp_name = os.path.join(root, name)
                if os.path.isfile(temp_name) and (selection_string in temp_name) and (selection_string2 in temp_name) and (avoid_string not in temp_name) :
                    filenamelist.append(os.path.join(root, name))
    filenamelist.sort()
    return filenamelist

def play_np_array(data, min=0, max=1, cmap = "jet"):
    matplotlib.rcParams['animation.embed_limit'] = 2**128
    
    fig, axs = plt.subplots()
    fig.set_figheight(15)
    fig.set_figwidth(15)
    im0 = axs.imshow(data[0], vmin = min, vmax =max, cmap = cmap )

    def init():
        im0.set_data(data[0])

    def animate(i):
        im0.set_data(data[i])

        return im0

    anim = animation.FuncAnimation(fig, animate, init_func=init, frames=data.shape[0], repeat = True)
    return HTML(anim.to_jshtml())

In [40]:
# This step  will take a long time

for day_of_year in tqdm(range(248,260)):
    for hour in range(24):
        bucket_name = 'noaa-goes17'
        product_name = 'GLM-L2-LCFA'
        year = 2020
        #day_of_year = 239
        #hour = 1
        #band = 8


        prefix = f'{product_name}/{year}/{day_of_year:03.0f}/{hour:02.0f}/OR_{product_name}'
        output_file_name = "lightning_map/"+prefix.replace("/","_")+".npy"
        if os.path.exists(output_file_name):
            continue
        else:
            keys = get_s3_keys(bucket_name,
                        s3_client,
                        prefix = prefix,
                        )
            #print(prefix)
            #print(day_of_year)

            all_events = []
            for key in keys:
                #print(key)
                resp = requests.get(f'https://{bucket_name}.s3.amazonaws.com/{key}')

                file_name = key.split('/')[-1].split('.')[0]
                nc4_ds = netCDF4.Dataset(file_name, memory = resp.content)
                store = xr.backends.NetCDF4DataStore(nc4_ds)
                DS = xr.open_dataset(store)
                C = DS
                all_events.append(DS)

                x_values = y_values = v_values = np.empty(0)
                for DS in all_events:
                    x_values = np.append(x_values, DS.event_lon.values)
                    y_values = np.append(y_values, DS.event_lat.values)
                    v_values = np.append(v_values, DS.event_energy.values)
            np.save(output_file_name, np.array([x_values, y_values, v_values]))


100%|██████████| 12/12 [00:00<00:00, 4936.41it/s]


In [41]:
filenamelist = load_file_names("GLM-L2-LCFA",".npy","xxx",["."])

In [42]:
filenamelist[:10]

['./lightening_map/GLM-L2-LCFA_2020_248_00_OR_GLM-L2-LCFA.npy',
 './lightening_map/GLM-L2-LCFA_2020_248_01_OR_GLM-L2-LCFA.npy',
 './lightening_map/GLM-L2-LCFA_2020_248_02_OR_GLM-L2-LCFA.npy',
 './lightening_map/GLM-L2-LCFA_2020_248_03_OR_GLM-L2-LCFA.npy',
 './lightening_map/GLM-L2-LCFA_2020_248_04_OR_GLM-L2-LCFA.npy',
 './lightening_map/GLM-L2-LCFA_2020_248_05_OR_GLM-L2-LCFA.npy',
 './lightening_map/GLM-L2-LCFA_2020_248_06_OR_GLM-L2-LCFA.npy',
 './lightening_map/GLM-L2-LCFA_2020_248_07_OR_GLM-L2-LCFA.npy',
 './lightening_map/GLM-L2-LCFA_2020_248_08_OR_GLM-L2-LCFA.npy',
 './lightening_map/GLM-L2-LCFA_2020_248_09_OR_GLM-L2-LCFA.npy']

In [45]:
import matplotlib.animation as animation
import matplotlib.cm as cm
from IPython.display import HTML
import matplotlib

import cv2

import cartopy.io.shapereader as shpreader
import cartopy.feature as cfeature

reader = shpreader.Reader('countyl010g_shp/countyl010g.shp')

counties = list(reader.geometries())

COUNTIES = cfeature.ShapelyFeature(counties, ccrs.PlateCarree())

img = [] # some array of images
frames = [] # for storing the generated images

for i,filename in enumerate(filenamelist):
  data = np.load(filename)
  x_values, y_values, v_values = data[0], data[1], data[2]
  mask = (x_values>-160) & (x_values<-80) & (y_values>0) & (v_values > 0)

  fig = plt.figure(figsize=(15, 12))
  pc = ccrs.PlateCarree()


  # Create axis with Geostationary projection
  ax = fig.add_subplot(1, 1, 1, projection=pc)
  ax.set_extent([-130,-110,30,50], crs=pc)

  # Add the RGB image to the figure. The data is in the same projection as the
  # axis we just created.
  #ax.imshow(DS.event_energy.values, origin='upper',
    #        extent=(xx.min(), xx.max(), yy.min(), yy.max()), transform=geos)
  plt.hist2d(x_values[mask], y_values[mask], bins = (100, 100), range = ([-130,-110],[30,50]),cmap=plt.cm.jet, cmin = 1, cmax=10)


  # Add Coastlines and States
  ax.coastlines(resolution='50m', color='black', linewidth=0.25)
  ax.add_feature(ccrs.cartopy.feature.STATES, linewidth=0.25)
  ax.add_feature(COUNTIES, facecolor='none', edgecolor='gray',linewidth=0.2)

  plt.xlim = ([-160,-80])

  plt.title('GOES-17 GLM (Lightning)', loc='left', fontweight='bold', fontsize=15)
  #plt.title('{}'.format(scan_start.strftime('%d %B %Y %H:%M UTC ')), loc='right')
  plt.title('{}'.format(filename), loc='right')
  plt.savefig("lightning_images/image"+str(i)+".jpg")
  plt.close()
  frames.append(cv2.imread("lightning_images/image"+str(i)+".jpg"))

np.save("lightning_images/all_frames.npy", frames)

In [ ]:
play_np_array(np.array(frames))